In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from itertools import combinations

In [2]:
#APERTURA ARCHIVOS
test = pd.read_pickle("Archivos/Arboles_validacion.pkl")
prediccion_arbol = pd.read_csv("Archivos/prediccion_arbol_validacion.csv")
prediccion_red   = pd.read_csv("Archivos/prediccion_neurona_validacion.csv")

In [3]:
test = test.sort_values('Opportunity_ID')
prediccion_arbol = prediccion_arbol.sort_values('Opportunity_ID')
prediccion_red   = prediccion_red.sort_values('Opportunity_ID')
test_label = test['Stage']

In [4]:
prediccion_red.count()

Opportunity_ID    4515
Target            4515
dtype: int64

In [5]:
prediccion_arbol.count()

Opportunity_ID    4515
Target            4515
dtype: int64

In [6]:
#MAX
prediccion = prediccion_arbol['Target'].combine(prediccion_red['Target'], max, 0)
skl.metrics.log_loss(test_label,prediccion)

0.9143752132190162

In [7]:
#MIN
prediccion = prediccion_arbol['Target'].combine(prediccion_red['Target'], min, 0)
skl.metrics.log_loss(test_label,prediccion)

1.255039262129048

In [8]:
#COMBINACION LINEAL
min_a = -1
min_b = -1
min_logloss = 1000

for a in range(0,100):
    for b in range(0,100):
        if(a == 0 & b == 0): continue
        prediccion = (a*prediccion_arbol['Target'] + b*prediccion_red['Target'])/(a+b)
        logloss = skl.metrics.log_loss(test_label,prediccion)
        if(logloss < min_logloss):
            min_logloss = logloss
            min_a = a
            min_b = b

In [9]:
print("min_logloss: ", min_logloss)
print("min_a: ", min_a)
print("min_b: ", min_b)

min_logloss:  0.7119746790443063
min_a:  39
min_b:  10


In [10]:
#KNN
temp = pd.DataFrame()
temp['Arbol'] = prediccion_arbol['Target']
temp['Red'] = prediccion_red['Target']
temp['Stage'] = test_label
entrenamiento = temp.iloc[:4000,:]
validacion = temp.iloc[4001:,:]
entrenamiento_label = entrenamiento.pop('Stage')
classifier = KNeighborsClassifier(n_neighbors=300)
classifier.fit(entrenamiento, entrenamiento_label)
preds = classifier.predict_proba(validacion[['Arbol','Red']])
print("logloss: ", skl.metrics.log_loss(validacion['Stage'],preds))

logloss:  0.16438261146424551


In [11]:
alto

NameError: name 'alto' is not defined

In [14]:
#FINAL COMBINACION LINEAL
prediccion_arbol = pd.read_csv("Archivos/prediccion_arbol_test.csv")
prediccion_red   = pd.read_csv("Archivos/prediccion_neurona_test.csv")
preds = (min_a*prediccion_arbol['Target'] + min_b*(1-prediccion_red['Target']))/(min_a+min_b)
preds

0       0.457069
1       0.457458
2       0.455983
3       0.458949
4       0.467020
          ...   
1562    0.453635
1563    0.454685
1564    0.454685
1565    0.433457
1566    0.438353
Name: Target, Length: 1567, dtype: float64

In [15]:
#FINAL KNN
classifier.fit(temp[['Arbol','Red']], temp['Stage'])
prediccion_arbol = pd.read_csv("Archivos/prediccion_arbol_test.csv")
prediccion_red   = pd.read_csv("Archivos/prediccion_neurona_test.csv")
fin = pd.DataFrame()
fin['Arbol'] = prediccion_arbol['Target']
fin['Red'] = prediccion_red['Target']
preds = classifier.predict_proba(fin)
preds = pd.Series([p[1] for p in preds])

In [16]:
preds

0       0.950000
1       0.946667
2       0.940000
3       0.946667
4       0.996667
          ...   
1562    0.946667
1563    0.926667
1564    0.926667
1565    0.030000
1566    0.030000
Length: 1567, dtype: float64

In [ ]:
alto

In [ ]:
#GUARDADO DE ARCHIVOS
resultados = pd.DataFrame()
resultados['Opportunity_ID'] = prediccion_arbol['Opportunity_ID']
resultados['Target'] = pd.Series(preds)  
resultados.to_csv("Archivos/prediccion.csv", index=False)